In [1]:
import numpy as np
import pandas as pd
import requests
import urllib.request
from bs4 import BeautifulSoup

In [2]:
base_url = "https://www.worldfootball.net/schedule/eng-premier-league-2018-2019-spieltag/"

In [85]:
team1 = [list(range(1,21))]
team2 = [list(range(1,21))]
colnames1 = ['Position']
colnames2 = ['Position']
#weekly_standing = pd.DataFrame(columns = ['Position', 'Team'])
for matchweek in range(1, 39):
    standing = []
    points = []
    colnames1.append("Team"+str(matchweek))
    colnames2.append("Points"+str(matchweek))
    url = base_url + str(matchweek)
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    data = soup.select('table.standard_tabelle')
    
    for team in data[1].findAll('a'):
        standing.append(team.text)
    
    for team in data[1].findAll('tr'):
        count = 0
        for rec in team.findAll('td'):
            if (count == 9):
                points.append(rec.text)
            count += 1
        
    team1.append(standing)
    team2.append(points)
#print(teams)
#print(colnames)

weekly_standing = pd.DataFrame.from_records(team1).T
weekly_standing.columns = colnames1

weekly_points = pd.DataFrame.from_records(team2).T
weekly_points.columns = colnames2

In [87]:
weekly_points

,Position,Points1,Points2,Points3,Points4,Points5,Points6,Points7,Points8,Points9,...,Points29,Points30,Points31,Points32,Points33,Points34,Points35,Points36,Points37,Points38
0,1,3,6,9,12,15,18,19,20,23,...,71,74,77,80,83,86,89,92,95,98
1,2,3,6,9,12,15,16,19,20,23,...,70,73,76,79,82,85,88,91,94,97
2,3,3,6,9,12,13,16,17,20,21,...,61,61,64,64,67,70,70,70,71,72
3,4,3,6,9,10,12,13,15,18,21,...,59,60,60,63,66,66,67,68,70,71
4,5,3,6,7,9,10,12,15,18,21,...,58,60,60,63,63,66,66,66,67,70
5,6,3,6,7,7,9,12,13,16,17,...,57,58,58,61,61,64,64,65,66,66
6,7,3,4,6,6,9,10,13,15,16,...,43,44,47,47,50,50,51,54,57,57
7,8,3,3,5,6,9,10,12,13,15,...,43,43,44,44,47,47,50,51,53,54
8,9,1,3,3,6,8,9,12,13,15,...,39,39,42,44,47,47,49,50,51,52
9,10,1,3,3,6,6,9,10,12,14,...,37,38,41,43,46,46,48,50,50,52


In [88]:
weekly_standing.to_csv(path_or_buf= "weekly_standing.csv",index=False)
weekly_points.to_csv(path_or_buf= "weekly_points.csv",index=False)

In [81]:
teams = [list(range(1,21))]
print(teams[0]

SyntaxError: invalid syntax (<ipython-input-81-8fcbd1427285>, line 2)

In [192]:
a = list(range(1,3))
b = list(['ManU', "ManC"])
c = pd.DataFrame(data=[a,b])

In [193]:
c

,0,1
0,1,2
1,ManU,ManC


In [169]:
teams

[('Position',
  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])]

'1'

In [55]:
url = "https://www.worldfootball.net/schedule/eng-premier-league-2018-2019-spieltag/38"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
    
data = soup.select('table.standard_tabelle')
points = []
for team in data[1].findAll('tr'):
    count = 0
    for rec in team.findAll('td'):
        if (count == 9):
            points.append(rec.text)
        count += 1

points

['98',
 '97',
 '72',
 '71',
 '70',
 '66',
 '57',
 '54',
 '52',
 '52',
 '50',
 '49',
 '45',
 '45',
 '40',
 '39',
 '36',
 '34',
 '26',
 '16']